In [ ]:
#Export code based on Corentin Chauvin
#https://github.com/CorentinChauvin/playlist-exporter

DeezerPLNo = 12345
SpotifyPLNO = 0
global CSVFilename 
CSVFilename= 'DPL'+str(DeezerPLNo)+'.csv'

print("Saving Deezer Playlist to: ", CSVFilename)

In [ ]:
#Extract tracks from Public Deezer Playlist
import requests

def load_playlist(playlist_id):
    STATUS_OK = 200
    base_url = "https://api.deezer.com/playlist/"
    playlist_id = str(playlist_id)
    tracks_req = "/tracks?index=0&limit={}"

    r = requests.get(base_url + playlist_id)

    if r.status_code != STATUS_OK:
        print("Status code: {}".format(r.status_code))
        return None

    data = r.json()

    nb_tracks = data['nb_tracks']
    songs = None

    if len(data['tracks']['data']) == nb_tracks:
        songs = data['tracks']['data']
    else:
        # By default, loads only 400 songs. To load everything, should use an other
        # kind of request
        r = requests.get(base_url + playlist_id + tracks_req.format(nb_tracks))
        songs = r.json()['data']


    # Create a string of tracks
    s = "Title,Artist,Album,Duration,Time added,Id song,Id artist,Id album\n\n"

    for song in songs:
        s += "\"{}\",\"{}\",\"{}\",{},{},{},{},{}\n".format(song['title'], song['artist']['name'], song['album']['title'],
            song['duration'], song['time_add'], song['id'], song['artist']['id'], song['album']['id'])

    return data, s

In [ ]:
#Save Tracklist as CSV-File
tracks_string = None

def save_pl():
    global deezer_pl_name
    global tracks_string

    data, tracks_string = load_playlist(DeezerPLNo)

    if data is not None:
        deezer_pl_name =data['title']
        print("Playlist name: {}     ".format(data['title']))
        print("No tracks: {}".format(data['nb_tracks']))

        print(tracks_string)
        if CSVFilename is not None:
          f = open(CSVFilename, 'w+')
          f.write(tracks_string)
          f.close()



In [ ]:
i = save_pl()

# Write new list to Spotify with spotipy
https://github.com/plamere/spotipy/tree/master/examples

In [ ]:
CLIENT_ID = "9ab44bco3849349384938943be8b2c63116c"  #add your id from developer.spotify
CLIENT_SEC = "3493849384938q0830983204"             #add your id from developer.spotify
REDIRECT_URI = "http://localhost:8000/"             #add create idleing webserver with  python3 -m http.server
username='yourspotifyusername'



In [ ]:
import sys
import spotipy
import spotipy.util as util

#AUTH

scope = 'playlist-modify-public user-library-read'

token=util.prompt_for_user_token(username,
                           scope,
                           client_id=CLIENT_ID,
                           client_secret=CLIENT_SEC,
                           redirect_uri=REDIRECT_URI)


if token:
    sp = spotipy.Spotify(auth=token)
  
else:
    print("Can't get token for", username)
    
   

In [ ]:
print("Adding ", deezer_pl_name, " to your Spotify Public Playlistst: ")

#Show all public playlists
results = sp.current_user_playlists(limit=50)
for i, item in enumerate(results['items']):
    print("%d %s %s" % (i, item['name'],item['id']))

In [ ]:
import csv
import re

OldList = []
Failed =[]

print ("Creating new Playlist ", deezer_pl_name)
print ("with tracks: ")
with open(CSVFilename) as csvfile:
        csvdata = csv.DictReader(csvfile)
        for row in csvdata:
            search_str= re.sub(r'\([^)]*\)', '', row['Title']) + ' ' +row['Artist']
            #print(search_str)
            results = sp.search(q=search_str, type='track')
            try:
              r =results['tracks']['items'][0]['id']   
            except IndexError:
              #print('Importing Playlist has failed for this track: ',search_str )
              Failed.append(search_str)
              r=""
            if r !="":  
                print(search_str)
                OldList.append(r)    
            
print('------')
print("Import has failed for the following tracks ")
print(', '.join(Failed) )

In [ ]:

user_id = sp.me()['id']
res = sp.user_playlist_create(user_id, deezer_pl_name)
id_newlist= res['id']
res2 = sp.user_playlist_add_tracks(user_id,id_newlist , OldList)



In [ ]:
#print(res2)
if (res2):
  print("Created new playlist for Spotify User", username, " with Title: ", deezer_pl_name)